# مروری کلی بر کرونا
> مروری بر کل موارد کرونا ، مرگ و میر و موارد جدید براساس کشور.

- comments: true
- author: پراتاب واردهان و سولماز عباسپور
- categories: [مرور]
- image: images/covid-overview.png
- permalink: /covid-overview/
- hide: false

In [1]:
#hide
print('''
Example of using jupyter notebook, pandas (data transformations), jinja2 (html, visual)
to create visual dashboards with fastpages
You see also the live version on https://gramener.com/enumter/covid19/
''')


Example of using jupyter notebook, pandas (data transformations), jinja2 (html, visual)
to create visual dashboards with fastpages
You see also the live version on https://gramener.com/enumter/covid19/



In [2]:
#hide
!pip install jdatetime
!pip install persiantools
import numpy as np
import pandas as pd
from jinja2 import Template
import jdatetime as jd
import os
from persiantools import digits
from IPython.display import HTML

You should consider upgrading via the '/opt/hostedtoolcache/Python/3.6.10/x64/bin/python -m pip install --upgrade pip' command.


You should consider upgrading via the '/opt/hostedtoolcache/Python/3.6.10/x64/bin/python -m pip install --upgrade pip' command.


In [3]:
#hide

# FETCH
base_url = 'https://raw.githubusercontent.com/pratapvardhan/notebooks/master/covid19/'
dirpath = os.path.dirname(os.getcwd())
paths = {
    'mapping': base_url + 'mapping_countries.csv',
    'overview': dirpath + '/_templates/overview.tpl'
}

def get_mappings(url):
    df = pd.read_csv(url)
    return {
        'df': df,
        'replace.country': dict(df.dropna(subset=['Name']).set_index('Country')['Name']),
        'map.continent': dict(df.set_index('Name')['Continent'])
    }

mapping = get_mappings(paths['mapping'])

def get_template(path):
    from urllib.parse import urlparse
    if bool(urlparse(path).netloc):
        from urllib.request import urlopen
        return urlopen(path).read().decode('utf8')
    return open(path).read()

def get_frame(name):
    url = (
        'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/'
        f'csse_covid_19_time_series/time_series_covid19_{name}_global.csv')
    df = pd.read_csv(url)
    # rename countries
    df['Country/Region'] = df['Country/Region'].replace(mapping['replace.country'])
    return df

def get_dates(df):
    dt_cols = df.columns[~df.columns.isin(['Province/State', 'Country/Region', 'Lat', 'Long'])]
    LAST_DATE_I = -1
    # sometimes last column may be empty, then go backwards
    for i in range(-1, -len(dt_cols), -1):
        if not df[dt_cols[i]].fillna(0).eq(0).all():
            LAST_DATE_I = i
            break
    return LAST_DATE_I, dt_cols

In [4]:
#hide
COL_REGION = 'Country/Region'
# Confirmed, Recovered, Deaths
df = get_frame('confirmed')
# dft_: timeseries, dfc_: today country agg
dft_cases = df
dft_deaths = get_frame('deaths')
dft_recovered = get_frame('recovered')
LAST_DATE_I, dt_cols = get_dates(df)

dt_today = dt_cols[LAST_DATE_I]
dt_5ago = dt_cols[LAST_DATE_I-5]


dfc_cases = dft_cases.groupby(COL_REGION)[dt_today].sum()
dfc_deaths = dft_deaths.groupby(COL_REGION)[dt_today].sum()
dfp_cases = dft_cases.groupby(COL_REGION)[dt_5ago].sum()
dfp_deaths = dft_deaths.groupby(COL_REGION)[dt_5ago].sum()

In [5]:
#hide
df_table = (pd.DataFrame(dict(Cases=dfc_cases, Deaths=dfc_deaths, PCases=dfp_cases, PDeaths=dfp_deaths))
             .sort_values(by=['Cases', 'Deaths'], ascending=[False, False])
             .reset_index())
df_table.rename(columns={'index': 'Country/Region'}, inplace=True)
for c in 'Cases, Deaths'.split(', '):
    df_table[f'{c} (+)'] = (df_table[c] - df_table[f'P{c}']).clip(0)  # DATA BUG
df_table['Fatality Rate'] = (100 * df_table['Deaths'] / df_table['Cases']).round(1)
df_table['Continent'] = df_table['Country/Region'].map(mapping['map.continent'])
df_table.head(15)

,Country/Region,Cases,Deaths,PCases,PDeaths,Cases (+),Deaths (+),Fatality Rate,Continent
0,US,1309550,78795,1180375,68922,129175,9873,6.0,North America
1,Spain,223578,26478,218011,25428,5567,1050,11.8,Europe
2,Italy,218268,30395,211938,29079,6330,1316,13.9,Europe
3,United Kingdom,216525,31662,191832,28809,24693,2853,14.6,Europe
4,Russia,198676,1827,145268,1356,53408,471,0.9,Europe
5,France,176782,26313,169583,25204,7199,1109,14.9,Europe
6,Germany,171324,7549,166152,6993,5172,556,4.4,Europe
7,Brazil,156061,10656,108620,7367,47441,3289,6.8,South America
8,Turkey,137115,3739,127659,3461,9456,278,2.7,Asia
9,Iran,106220,6589,98647,6277,7573,312,6.2,Asia


In [6]:
#hide
#delete problematic countries from table
df_table = df_table[~df_table['Country/Region'].isin(['Cape Verde', 'Cruise Ship', 'Kosovo'])]

In [7]:
#hide
# world, china, europe, us
metrics = ['Cases', 'Deaths', 'Cases (+)', 'Deaths (+)']
s_china = df_table[df_table['Country/Region'].eq('China')][metrics].sum().add_prefix('China ')
s_us = df_table[df_table['Country/Region'].eq('US')][metrics].sum().add_prefix('US ')
s_eu = df_table[df_table['Continent'].eq('Europe')][metrics].sum().add_prefix('EU ')
s_iran = df_table[df_table['Country/Region'].eq('Iran')][metrics].sum().add_prefix('Iran ')
summary = {'updated': pd.to_datetime(dt_today), 'since': pd.to_datetime(dt_5ago)}
summary = {**summary, **df_table[metrics].sum(), **s_china, **s_us, **s_eu, **s_iran}
summary

{'updated': Timestamp('2020-05-09 00:00:00'),
 'since': Timestamp('2020-05-04 00:00:00'),
 'Cases': 4023147,
 'Deaths': 279283,
 'Cases (+)': 443757,
 'Deaths (+)': 27772,
 'China Cases': 83990,
 'China Deaths': 4637,
 'China Cases (+)': 24,
 'China Deaths (+)': 0,
 'US Cases': 1309550,
 'US Deaths': 78795,
 'US Cases (+)': 129175,
 'US Deaths (+)': 9873,
 'EU Cases': 1566888,
 'EU Deaths': 151792,
 'EU Cases (+)': 127025,
 'EU Deaths (+)': 9627,
 'Iran Cases': 106220,
 'Iran Deaths': 6589,
 'Iran Cases (+)': 7573,
 'Iran Deaths (+)': 312}

In [8]:
#hide
dft_ct_cases = dft_cases.groupby(COL_REGION)[dt_cols].sum()
dft_ct_new_cases = dft_ct_cases.diff(axis=1).fillna(0).astype(int)
dft_ct_new_cases.head()

,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,1/30/20,1/31/20,...,4/30/20,5/1/20,5/2/20,5/3/20,5/4/20,5/5/20,5/6/20,5/7/20,5/8/20,5/9/20
Country/Region,,,,,,,,,,,,,,,,,,,,,
Afghanistan,0,0,0,0,0,0,0,0,0,0,...,232,164,134,235,190,330,168,171,215,255
Albania,0,0,0,0,0,0,0,0,0,0,...,7,9,7,6,8,17,12,10,8,6
Algeria,0,0,0,0,0,0,0,0,0,0,...,158,148,141,179,174,190,159,185,187,189
Andorra,0,0,0,0,0,0,0,0,0,0,...,2,0,2,1,2,1,0,1,0,2
Angola,0,0,0,0,0,0,0,0,0,0,...,0,3,5,0,0,1,0,0,7,0


In [9]:
#hide_input
template = Template(get_template(paths['overview']))
days = digits.en_to_fa(str((summary['updated'] - summary['since']).days))
g_date_updated = pd.to_datetime(summary['updated'])
jdate_updated = jd.date.fromgregorian(year=g_date_updated.year,month=g_date_updated.month,day=g_date_updated.day,locale='fa_IR').strftime("%d / %m / %Y")
jdate_updated = digits.en_to_fa(jdate_updated)
g_date_since = pd.to_datetime(summary['since'])
jdate_since = jd.date.fromgregorian(year=g_date_since.year,month=g_date_since.month,day=g_date_since.day,locale='fa_IR').strftime("%d / %m / %Y")
jdate_since = digits.en_to_fa(jdate_since)
html = template.render(
    D=summary, table=df_table,  # REMOVE .head(20) to see all values
    newcases=dft_ct_new_cases.loc[:, dt_cols[LAST_DATE_I-50]:dt_cols[LAST_DATE_I]],
    np=np, pd=pd, days=days, digits=digits, 
    jdate_updated=jdate_updated, jdate_since=jdate_since, enumerate=enumerate)
HTML(f'<div>{html}</div>')


تحلیل اصلی از [پراتاپ واردهان](https://twitter.com/PratapVardhan)[^۱]

[^۱]: منبع: ["2019 Novel Coronavirus COVID-19 (2019-nCoV) Data Repository by Johns Hopkins CSSE"](https://systems.jhu.edu/research/public-health/ncov/) [GitHub repository](https://github.com/CSSEGISandData/COVID-19). لینک به صفحه [اصلی](https://github.com/pratapvardhan/notebooks/blob/master/covid19/covid19-compare-country-trajectories.ipynb).